In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time
import re

# Начальная и конечная дата
start_date = datetime(2022, 9, 1)
end_date = datetime(2022, 9, 10) # протянуть дату до 31 декабря

# Список для хранения данных
data = []

# Перебираем даты
current_date = start_date
while current_date <= end_date:
    url = f"https://www.meteoservice.ru/archive/moskva/{current_date.year}/{current_date.strftime('%m')}/{current_date.strftime('%d')}"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ищем div с нужным классом
        callout_div = soup.find('div', class_='callout', attrs={'data-equalizer-watch': True})
        if callout_div:
            paragraphs = callout_div.find_all('p')
            if len(paragraphs) >= 2:
                weather_text = paragraphs[1].get_text(strip=True)
            else:
                weather_text = "Нет второго параграфа"
        else:
            weather_text = "Нет блока callout"
            
    except Exception as e:
        weather_text = f"Ошибка: {e}"
    
    # Сохраняем дату и текст
    data.append({
        "date": current_date.strftime('%Y-%m-%d'),
        "weather": weather_text
    })
    
    # Переходим к следующему дню
    current_date += timedelta(days=1)
    
    time.sleep(1)  # Пауза 1 секунда


In [2]:

# Функция парсинга
def parse_weather(text):
    # Давление    
    pressure_match = re.search(r'Давление было (.*?)(\.|\n)', text)
    pressure = pressure_match.group(1).strip() if pressure_match else None
    # Температура
    temp_match = re.search(r'Температура воздуха ([\+\-0-9…°C]+)', text)
    if temp_match:
        temps = re.findall(r'[\+\-]?\d+', temp_match.group(1))
        if len(temps) == 2:
            avg_temp = (int(temps[0]) + int(temps[1])) / 2
        elif len(temps) == 1:
            avg_temp = int(temps[0])
        else:
            avg_temp = None
    else:
        avg_temp = None
    
    # Ветер
    wind_match = re.search(r'Ветер .*? ([0-9]+)—([0-9]+) м/с', text)
    if wind_match:
        wind_speed = (int(wind_match.group(1)) + int(wind_match.group(2))) / 2
    else:
        wind_speed = None
    
    # Влажность
    humidity_match = re.search(r'Влажность воздуха ([0-9]+)—([0-9]+)%', text)
    if humidity_match:
        humidity = (int(humidity_match.group(1)) + int(humidity_match.group(2))) / 2
    else:
        humidity = None
        # Осадки (любые)
    if 'осадки' in text or 'дождь' in text or 'снег' in text:
        precipitation = 'Были осадки'
    else:
        precipitation = 'Без осадков'
    
    # Дождь отдельно
    if 'дождь' in text:
        rain = 'Был дождь'
    else:
        rain = 'Не было дождя'
    
    return pd.Series({
        'pressure': pressure,
        'avg_temp': avg_temp,
        'avg_wind': wind_speed,
        'avg_humidity': humidity,
        'precipitation': precipitation,
        'rain': rain
    })

df_weather = pd.DataFrame(data)
# Применяем к DataFrame
df_weather[['pressure', 'avg_temp', 'avg_wind', 'avg_humidity', 'precipitation', 'rain']] = df_weather['weather'].apply(parse_weather)

# Посмотрим что получилось
# print(df_weather[['date', 'avg_temp', 'avg_wind', 'avg_humidity']])

# Считаем средние значения
mean_temp = df_weather['avg_temp'].mean()
mean_wind = df_weather['avg_wind'].mean()
mean_humidity = df_weather['avg_humidity'].mean()

print(f"Средняя температура: {mean_temp:.2f} °C")
print(f"Средняя скорость ветра: {mean_wind:.2f} м/с")
print(f"Средняя влажность: {mean_humidity:.2f} %")



Средняя температура: 10.30 °C
Средняя скорость ветра: 1.25 м/с
Средняя влажность: 68.50 %


In [3]:
df_weather.head(10)

,date,weather,pressure,avg_temp,avg_wind,avg_humidity,precipitation,rain
0,2022-09-01,Днем 1 сентября 2022 года в Москве характер по...,ниже нормы,11.5,1.0,69.0,Были осадки,Не было дождя
1,2022-09-02,Днем 2 сентября 2022 года в Москве характер по...,в пределах нормы,9.5,1.0,73.5,Были осадки,Был дождь
2,2022-09-03,Днем 3 сентября 2022 года в Москве характер по...,в пределах нормы,12.5,1.0,63.0,Без осадков,Не было дождя
3,2022-09-04,Днем 4 сентября 2022 года в Москве характер по...,в пределах нормы,11.5,1.0,58.0,Без осадков,Не было дождя
4,2022-09-05,Днем 5 сентября 2022 года в Москве характер по...,в пределах нормы,10.0,1.5,74.5,Были осадки,Не было дождя
5,2022-09-06,Днем 6 сентября 2022 года в Москве характер по...,в пределах нормы,9.5,2.0,72.0,Были осадки,Был дождь
6,2022-09-07,Днем 7 сентября 2022 года в Москве характер по...,ниже нормы,10.0,0.5,76.5,Были осадки,Был дождь
7,2022-09-08,Днем 8 сентября 2022 года в Москве характер по...,в пределах нормы,8.5,3.0,74.0,Были осадки,Был дождь
8,2022-09-09,Днем 9 сентября 2022 года в Москве характер по...,в пределах нормы,9.0,1.0,64.5,Без осадков,Не было дождя
9,2022-09-10,Днем 10 сентября 2022 года в Москве характер п...,в пределах нормы,11.0,0.5,60.0,Без осадков,Не было дождя


### Сохранение данных

In [ ]:
# df_weather.to_csv('data_weather.csv', index=False, encoding='utf-8-sig')